<a href="https://colab.research.google.com/github/Ousmane225/Tp-Spark/blob/master/Boxes_Computation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Boxes Computation

In this exercise, you will use the TensorFlow object detection API to get bounding boxes and classes on images.

But first, we need some installation.

## I. Installation

We will here follow the installation guide of the API, that can be found [here](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/installation.md).

### I.1. Dependencies

The first thing to do is to install all needed dependencies (if not installed yet):
- pip install --user Cython
- pip install --user contextlib2
- pip install --user pillow
- pip install --user lxml
- pip install --user jupyter
- pip install --user matplotlib



### I.2. Models

Now we will download the models (i.e. the architecture and trained weights of neural networks). They are available in the so called [detection model zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md).

For this exercise, we will use first the **faster_rcnn_inception_v2_coco** model: download it.

You should get a .tar.gz file, containing (among other files) `frozen-inference-graph.pb`: this is what we will use to perform object detection.

So extract it in the `data/models` folder.

In [0]:
!curl http://download.tensorflow.org/models/object_detection/faster_rcnn_inception_v2_coco_2018_01_28.tar.gz --output faster_rcnn_inception_v2_coco.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  142M  100  142M    0     0   154M      0 --:--:-- --:--:-- --:--:--  153M


In [0]:
!tar xvzf faster_rcnn_inception_v2_coco.tar.gz

faster_rcnn_inception_v2_coco_2018_01_28/
faster_rcnn_inception_v2_coco_2018_01_28/model.ckpt.index
faster_rcnn_inception_v2_coco_2018_01_28/checkpoint
faster_rcnn_inception_v2_coco_2018_01_28/pipeline.config
faster_rcnn_inception_v2_coco_2018_01_28/model.ckpt.data-00000-of-00001
faster_rcnn_inception_v2_coco_2018_01_28/model.ckpt.meta
faster_rcnn_inception_v2_coco_2018_01_28/saved_model/
faster_rcnn_inception_v2_coco_2018_01_28/saved_model/saved_model.pb
faster_rcnn_inception_v2_coco_2018_01_28/saved_model/variables/
faster_rcnn_inception_v2_coco_2018_01_28/frozen_inference_graph.pb


We also need to clone the `models` part of the TensorFlow Object Detection API. To do so, open your terminal, and in the **root of the vivadata folder**, clone the repo with the following command:

```
git clone https://github.com/tensorflow/models.git
```

In [0]:
!git clone https://github.com/tensorflow/models.git

Cloning into 'models'...
remote: Enumerating objects: 8, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 34088 (delta 2), reused 6 (delta 0), pack-reused 34080
Receiving objects: 100% (34088/34088), 512.27 MiB | 34.57 MiB/s, done.
Resolving deltas: 100% (21849/21849), done.
Checking out files: 100% (2499/2499), done.


Finally, **do not commit those files**

### I.3. Protobuf

Protobuf (for protocol buffer) is a Google system, that will be used for configuration.

Go now in the newly cloned repo at the root of the vivadata folder `models/research`, and launch the configuration using protobuf:
```
protoc object_detection/protos/*.proto --python_out=.
```

You may need to install the protobuf compiler using the following command on Ubuntu: `sudo apt-get install protobuf-compiler`

For MacOS use `brew install protobuf`.


In [0]:
!sudo apt-get install protobuf-compiler

Reading package lists... Done
Building dependency tree       
Reading state information... Done
protobuf-compiler is already the newest version (3.0.0-9.1ubuntu1).
The following package was automatically installed and is no longer required:
  libnvidia-common-430
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 25 not upgraded.


In [0]:
%cd models/research/

/content/models/research


In [0]:
!protoc object_detection/protos/*.proto --python_out=.

In [0]:
%cd /content/

/content


## II. Object Detection

### II.1. Setting the paths to the trained graph

First, we set the paths of the model we will use in a variable called `PATH_TO_CKPT`: this is the path to the `frozen_inference_graph.pb` that you downloaded in I.2.

In [0]:
### TODO: define the variable PATH_TO_CKPT
PATH_TO_CKPT = "faster_rcnn_inception_v2_coco_2018_01_28/frozen_inference_graph.pb"

# Nouvelle section

Next you have to set the paths to the labels: indeed labels are just numbers, but we want them to be strings so that we can understand! The table to do so is in the folder you cloned: `models/research/object_detection/data/mscoco_label_map.pbtxt`.

Put that path into the variable `PATH_TO_LABELS`:

In [0]:
### TODO: define the variable PATH_TO_LABELS
PATH_TO_LABELS = "models/research/object_detection/data/mscoco_label_map.pbtxt"

Have a look at this file. How many classes are there? Put that value into a variable called `NUM_CLASSES`

In [0]:
### TODO: define the variable NUM_CLASSES
with open(PATH_TO_LABELS) as file:
    txt = file.read()
NUM_CLASSES = txt.count("display_name")
NUM_CLASSES

80

### II.2. Testing object detection

We made some utils functions for you, so that you will just have to put them together to do the object detection.

First, with the following code, you will compute the graph with the trained weights you downloaded:

In [0]:
import tensorflow.compat.v1 as tf

# Compute the graph
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

Then, you will have to use the functions `run_inference_for_single_image` provided in the `utils.py` file. This function is easy to find on the TensorFlow Object Detection API. Have a look at it and try to understand the big picture.

Then use it on the provided test images: `image1.jpg`, `image2` and `image3.jpg`.

In [0]:
### TODO: Use run_inference_for_single_image to compute the object detection


Now have a look at the output dictionary, can you understand its content? Save them in pickle format, in the next part of the challenge we will display and post process them!

In [0]:
### TODO: Save the output dicts in pickle
